**Part** **2**

### **Library**

In [ ]:
#%tensorflow_version 1.x
import xml.etree.ElementTree as ET
import numpy as np
import cv2
import glob
from google.colab.patches import cv2_imshow
import tensorflow as tf

from keras.layers import Input,Dense,Activation,Conv2D,AveragePooling2D,Flatten
from keras.models import Model 
from tensorflow.keras.utils import to_categorical

from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
import matplotlib as plt
%matplotlib inline


**Neural** **Network**

In [ ]:
images=glob.glob('/content/drive/MyDrive/data/images/*.jpg')
images.sort()
xmls=glob.glob('/content/drive/MyDrive/data/annotations/*.xml')
xmls.sort()

I = []
label_img = []

for xml_file in xmls:
    print("xml_file")
    print(xml_file)
    image_file = xml_file.replace('.xml','.jpg')
    image_file = image_file.replace('annotations','images')
    print("image_file")
    print(image_file)
    print("-----------------------------")
    I1 = cv2.imread(image_file)

    data_file = xml_file
    tree = ET.parse(data_file)
    root = tree.getroot()
    i = 0
    for x in root.iter('object'):
        i = i + 1
        label = x.find('name').text
        xmin = x.find('bndbox/xmin').text
        ymin = x.find('bndbox/ymin').text
        xmax = x.find('bndbox/xmax').text
        ymax = x.find('bndbox/ymax').text
        
        #print("---------------------" )
        #print("lable " + label)
        #print("xmin " + xmin)
        #print("ymin " + ymin)
        #print("xmax " + xmax)
        #print("ymax " + ymax)

        crop_img = I1[((int)(ymin)):((int)(ymax)), ((int)(xmin)):((int)(xmax))].copy()
        #print(crop_img.shape)
        dim = (32,35)
        crop_img = cv2.resize(crop_img, dim, interpolation = cv2.INTER_AREA)

        #label_list:
        if label== 'without_mask':
            label_img.append(0)
        if label== 'with_mask':
            label_img.append(1)
        if label== 'mask_weared_incorrect':
            label_img.append(2)
        I.append(crop_img)
        if i % 50 == 0:
          print(str(i) + "th epoch")
#convert list to array
label_img = np.array(label_img)
I = np.array(I)  

In [ ]:
#test and train:

x_train, x_test, y_train, y_test  = train_test_split(I, label_img , test_size=0.2, shuffle=True, random_state=5)

# #normalization:
# x_train = x_train.astype('float32')/255
# x_test = x_test.astype('float32')/255

print("x_train shape is {}".format( x_train.shape ))
print("y_train shape is {}".format( y_train.shape ))
print("x_test shape is {}".format( x_test.shape ))
print("y_test shape is {}".format( y_test.shape ))



In [ ]:
a = [0,0,0]
for i in range(x_train.shape[0]):
    a[0] = a[0] + x_train[i].shape[0]
    a[1] = a[1] + x_train[i].shape[1]

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 64
target_size = (50, 50)

train_datagen = ImageDataGenerator( rotation_range=10,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.1,
                                    zoom_range=0.1,
                                    channel_shift_range=0.0,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)

validation_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size)

In [ ]:
def build_model(input_shape):
  x_input = Input(shape =input_shape,name = 'input')

  x = Conv2D(filters = 8,kernel_size = (5,5), strides = 1, padding = 'valid',name = 'conv1')(x_input)
  x = Activation('relu')(x)
  x = AveragePooling2D(pool_size =5,strides = 2,name = 'pad1')(x)


  x = Conv2D(filters = 16,kernel_size = (5,5), strides = 1, padding = 'valid',name = 'conv2')(x)
  x = Activation('relu')(x)
  x = AveragePooling2D(pool_size =3,strides = 2,name = 'pad2')(x)


  x = Conv2D(filters = 24,kernel_size = (5,5), strides = 1, padding = 'valid',name = 'conv3')(x)
  x = Activation('relu')(x)
  x = AveragePooling2D(pool_size =3,strides = 2,name = 'pad3')(x)

  x = Flatten()(x)

  x = Dense(units = 80, name = 'fc_1')(x)

  x = Activation('relu', name = 'relu_1')(x)
  # x = Dropout(rate = 0.5)

  x = Dense(units = 50, name = 'fc_2')(x)
  x = Activation('relu', name = 'relu_2')(x)
  # x = Dropout(rate = 0.5)


  x = Dense(units = 40, name = 'fc_3')(x)
  x = Activation('relu', name = 'relu_3')(x)
  # x = Dropout(rate = 0.5)


  outputs = Dense(units =3,name = 'softmax', activation='softmax')(x)
  
  model = Model(inputs = x_input, outputs = outputs)
  model.summary()

  return model

In [ ]:
model = build_model(input_shape=(50,50,3))
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy' ,metrics = ['accuracy'])

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 30, 35, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 26, 31, 8)         608       
                                                                 
 activation_25 (Activation)  (None, 26, 31, 8)         0         
                                                                 
 pad1 (AveragePooling2D)     (None, 13, 16, 8)         0         
                                                                 
 conv2 (Conv2D)              (None, 9, 12, 16)         3216      
                                                                 
 activation_26 (Activation)  (None, 9, 12, 16)         0         
                                                                 
 pad2 (AveragePooling2D)     (None, 5, 6, 16)          0   

In [ ]:
datagen = ImageDataGenerator(
          rotation_range=30,
          width_shift_range=0.2,
          height_shift_range=0.2,
          zoom_range=0.2,
          horizontal_flip=True,
          fill_mode='nearest')

batch_size= 64
# model.fit(x = x_train, y= y_train , batch_size=64, epochs = 10)
results = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
  validation_data=(x_test, y_test),                       
	steps_per_epoch=len(y_train) // batch_size, epochs=250)